In [116]:
import pandas as pd
# import numpy as np
import requests as req
import sys
import time
# pip install holidays if you don't have the package already
import holidays
from datetime import datetime
from datetime import timedelta
# change the following filepath, file, and key name to your own
sys.path.insert(0, '../../API_Keys/')
from vantage_api import alpha_vantage_key

In [2]:
#below, be sure to change the name of 'apikey' to your key's name
def pull_all_stocks(df):
    stock_dict = {}
    for ticker in list(df['ticker'].unique()):
        json = req.get(url = "https://www.alphavantage.co/query",
                  params = {
                      'function': 'TIME_SERIES_DAILY',
                      'outputsize': 'full',
                      'symbol': ticker,
                      'apikey':alpha_vantage_key #change the name of the key to your key name
                  }).json()
        ticker_df = pd.DataFrame(json['Time Series (Daily)']).T
        ticker_df.rename(columns = {'1. open': 'open',
                                 '2. high': 'high',
                                 '3. low': 'low',
                                 '4. close': 'close',
                                 '5. volume': 'volume'}, inplace = True)
        if ticker_df.shape[0] > 2500:
            ticker_df = ticker_df.iloc[range(2500)] #Data beyond 2012 is unnecessary, this helps reduce file size
        stock_dict[f'{ticker}'] = ticker_df
        time.sleep(2) #Might need to be changed for long list of tickers
    return stock_dict


In [5]:
reps = pd.read_csv('data/cleaned_complete_congress_data.csv')

In [9]:
reps.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
1,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_partial,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
2,2022-05-31,X,"United States Steel Corporation <div class=""te...",sale_full,50000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
3,2022-05-31,X,United States Steel Corporation,purchase,250000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL
4,2022-05-20,PYPL,"PayPal Holdings, Inc. -",purchase,100000,Thomas H Tuberville,2022-06-13,senate,Thomas,Tuberville,Republican,1954-09-18,M,AL


In [130]:
mcconnell = reps[reps['last_name'] == 'McConnell']
mcconnell.head()

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents
0,2022-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-06-13,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
111,2022-03-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2022-03-14,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
355,2021-12-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-12-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
455,2021-09-03,WFC,WFC - Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-09-15,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY
669,2021-06-03,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2021-06-07,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY


In [110]:
mcconnell.shape

(23, 14)

In [ ]:
stocks = pd.DataFrame(columns = ['ticker','daily_time_series'])
time_series_df.set_index('ticker', inplace = True)

In [12]:
test = pull_all_stocks(mcconnell)

In [13]:
test

{'WFC':                open     high      low    close    volume
 2022-07-22  43.1700  43.5300  42.6900  43.1700  15965371
 2022-07-21  42.4800  43.3700  42.2800  43.2800  17808953
 2022-07-20  42.5400  43.0250  42.3550  42.7200  19111193
 2022-07-19  41.8800  43.0700  41.7450  42.9000  25724214
 2022-07-18  41.9300  42.7900  41.0200  41.1900  17991317
 ...             ...      ...      ...      ...       ...
 2012-08-21  34.2400  34.7600  34.1000  34.3800  25280400
 2012-08-20  34.0300  34.2000  33.9800  34.0700  18147600
 2012-08-17  34.2500  34.3000  33.9600  34.0300  16927000
 2012-08-16  34.1200  34.2000  33.7800  34.1300  19596400
 2012-08-15  34.0100  34.1800  33.8300  33.9700  13766800
 
 [2500 rows x 5 columns],
 'VMC':                 open      high       low     close  volume
 2022-07-22  157.1600  157.8200  154.1300  155.6500  480375
 2022-07-21  153.8600  156.6500  153.1450  156.4600  592335
 2022-07-20  151.5200  154.7800  151.5200  154.5800  632510
 2022-07-19  150.6100 

In [48]:
test['IR'].loc['2022-07-22'].astype(float)

open           45.00
high           45.19
low            44.18
close          44.54
volume    2200156.00
Name: 2022-07-22, dtype: float64

In [57]:
mcconnell[mcconnell['transaction_date'] == '2019-09-01']

,transaction_date,ticker,asset_description,type,amount,name,disclosure_date,chamber,first_name,last_name,party,birthday,gender,represents,price
2388,2019-09-01,WFC,Wells Fargo Company,purchase,15000,A. Mitchell Mcconnell,2019-09-16,senate,Mitchell,McConnell,Republican,1942-02-20,M,KY,NaN


In [60]:
test['WFC'].loc['2012-08-21']

open       34.2400
high       34.7600
low        34.1000
close      34.3800
volume    25280400
Name: 2012-08-21, dtype: object

# Transaction Date = Day Market Was Closed
I discovered there are dates that are in the DF that the market wasn't open, specifically 2019-09-01. building a try except now. [This stackoverflow](https://stackoverflow.com/a/61964191) helped with setting up the try/except. [This medium article](https://medium.com/analytics-vidhya/python-how-to-automate-bank-holidays-767de0d42d44) showed how us the holidays module. [This article](https://www.geeksforgeeks.org/python-holidays-library/) helped understand how to use it. [This](https://pynative.com/python-get-the-day-of-week/) showed how to get day of week to check if saturday or sunday so we can adjust day of transaction accordingly.

WRINKLE --> long weekends. gotta write for that eventuality.

We presume the holiday transaction date means that the orders were executed the next market day.

In [ ]:
#for example, this doesn't exist because 2019-09-1 was a Sunday so the market wasn't open
#test['WFC'].loc['2019-09-01']

In [88]:
us_holidays = holidays.UnitedStates()
type(holidays)

module

In [91]:
'2019-01-01' in us_holidays

True

In [87]:
us_holidays.items()

dict_items([(datetime.date(2019, 1, 1), "New Year's Day"), (datetime.date(2019, 1, 21), 'Martin Luther King Jr. Day'), (datetime.date(2019, 2, 18), "Washington's Birthday"), (datetime.date(2019, 5, 27), 'Memorial Day'), (datetime.date(2019, 7, 4), 'Independence Day'), (datetime.date(2019, 9, 2), 'Labor Day'), (datetime.date(2019, 10, 14), 'Columbus Day'), (datetime.date(2019, 11, 11), 'Veterans Day'), (datetime.date(2019, 11, 28), 'Thanksgiving'), (datetime.date(2019, 12, 25), 'Christmas Day')])

In [78]:
pd.DataFrame(usholidays)

""


In [105]:
# mcconnell['transaction_date'] = mcconnell['transaction_date'].map(pd.to_datetime(mcconnell['transaction_date']))
                                          

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_17438/267470481.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcconnell['transaction_date'] = mcconnell['transaction_date'].map(pd.to_datetime(mcconnell['transaction_date']))


In [111]:
mcconnell.dtypes

transaction_date     object
ticker               object
asset_description    object
type                 object
amount                int64
name                 object
disclosure_date      object
chamber              object
first_name           object
last_name            object
party                object
birthday             object
gender               object
represents           object
dtype: object

In [135]:
us_holidays = holidays.UnitedStates()
error_dates = []
holiday_dates = []
weird_holidays = [] #holidays for which market wasn't open the next day
saturdays = []
sundays = []
mcconnell['trading_day'] = 1
for i in mcconnell.index:
    try:
        mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']].drop('volume').astype(float).mean()
    except:
        trans_date = mcconnell.loc[i, 'transaction_date']
        trans_date_datetime = pd.to_datetime(mcconnell.loc[i, 'transaction_date'])
        trans_date = datetime(mcconnell.loc[i, 'transaction_date'], format = '%Y-%m-%d')
        print(trans_date)
        error_dates.append(trans_date)
        mcconnell.loc[i, 'trading_day'] = 0
        print(trans_date)
#BELOW HERE I NEED TO WORK OUT GETTING THE NEW DATE TO USE TO CALL UP THE PRICE I WANT
#THE ISSUE IS THAT THE DICTIONARY KEYS ARE STRINGS, SO I HAVE TO GET THE NEW DATE,
#THEN TURN IT INTO A YEAR-MO-DAY STRINGS
# MAYBE DO A DATE + TIMEDELTA AS A NEW VARIABLE WITHIN IF STATEMENT, THEN CONVERT/CALL THAT
        
        if trans_date in us_holidays:
            holiday_dates.append(trans_date)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+ timedelta(days = 1)].drop('volume').astype(float).mean()
            except:
                weird_holidays.append(trans_date)
        else:
            #check day of week, if saturday add 2 days, if sunday add 1 day
            if datetime.weekday(trans_date) == 5:
                saturdays.append(trans_date)
                if trans_date + timedelta(days = 2) not in us_holidays:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[trans_date + timedelta(days = 2)].drop('volume').astype(float).mean()
                else:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[trans_date + timedelta(days = 3)].drop('volume').astype(float).mean()
            # was here turing transaction to datetime in line before switching strategy
            elif datetime.weekday(trans_date) == 6:
                sundays.append(trans_date)
                if trans_date + timedelta(days = 1) not in us_holidays:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[trans_date+ timedelta(days = 1)].drop('volume').astype(float).mean()
                else:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[trans_date + timedelta(days = 2)].drop('volume').astype(float).mean()
    pass

/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_17438/913815542.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcconnell['trading_day'] = 1


ValueError: invalid literal for int() with base 10: '2019-09-01'

In [118]:
us_holidays = holidays.UnitedStates()
error_dates = []
holiday_dates = []
weird_holidays = [] #holidays for which market wasn't open the next day
saturdays = []
sundays = []
mcconnell['trading_day'] = 1
for i in mcconnell.index:
    try:
        mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']].drop('volume').astype(float).mean()
    except:
        trans_date = pd.to_datetime(mcconnell.loc[i, 'transaction_date'])
        error_dates.append(trans_date)
        mcconnell.loc[i, 'trading_day'] = 0
        print(trans_date)
        if trans_date in us_holidays:
            holiday_dates.append(trans_date)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+ timedelta(days = 1)].drop('volume').astype(float).mean()
            except:
                weird_holidays.append(trans_date)
        else:
            #check day of week, if saturday add 2 days, if sunday add 1 day
            if datetime.weekday(trans_date) == 5:
                saturdays.append(trans_date)
                if trans_date + timedelta(days = 2) not in us_holidays:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 2)].drop('volume').astype(float).mean()
                else:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[pd.to_datetime(mcconnell.loc[i, 'transaction_date']) + timedelta(days = 3)].drop('volume').astype(float).mean()
            # was here turing transaction to datetime in line before switching strategy
            elif datetime.weekday(trans_date) == 6:
                sundays.append(trans_date)
                if trans_date + timedelta(days = 1) not in us_holidays:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+ timedelta(days = 1)].drop('volume').astype(float).mean()
                else:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date'] + timedelta(days = 2)].drop('volume').astype(float).mean()
    pass

2019-09-01 00:00:00


/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_17438/3781565328.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcconnell['trading_day'] = 1


TypeError: can only concatenate str (not "datetime.timedelta") to str

In [108]:
error_dates

[NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT,
 NaT]

In [97]:
us_holidays = holidays.UnitedStates()
error_dates = []
holiday_dates = []
weird_holidays = [] #holidays for which market wasn't open the next day
saturdays = []
sundays = []
mcconnell['trading_day'] = 1
for i in mcconnell.index:
    try:
        mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']].drop('volume').astype(float).mean()
    except:
        trans_date = mcconnell.loc[i, 'transaction_date']
        error_dates.append(trans_date)
        mcconnell.loc[i, 'trading_day'] = 0
        print(trans_date)
        if trans_date in us_holidays:
            holiday_dates.append(trans_date)
            try:
                #trying the following day.
                mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+1].drop('volume').astype(float).mean()
            except:
                weird_holidays.append(trans_date)
        else:
            #check day of week, if saturday add 2 days, if sunday add 1 day
            if datetime.weekday(pd.to_datetime(trans_date)) == 5:
                saturdays.append(trans_date)
                if trans_date+2 not in us_holidays:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+2].drop('volume').astype(float).mean()
                else:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+3].drop('volume').astype(float).mean()
            elif datetime.weekday(pd.to_datetime(trans_date)) == 6:
                sundays.append(trans_date)
                if trans_date+1 not in us_holidays:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+1].drop('volume').astype(float).mean()
                else:
                    mcconnell.loc[i, 'price'] = test[mcconnell.loc[i, 'ticker']].loc[mcconnell.loc[i, 'transaction_date']+2].drop('volume').astype(float).mean()
    pass

2019-09-01


/var/folders/5_/498w081d1837cc82dmb4hwmc0000gn/T/ipykernel_17438/1450084766.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mcconnell['trading_day'] = 1


TypeError: can only concatenate str (not "int") to str

In [65]:
error_dates

['2019-09-01', '2019-06-01', '2018-12-05', '2017-02-02', '2016-06-09']

[Stackoverflow](https://stackoverflow.com/a/61426301) on mean in multipe columns

In [31]:
test[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']]

open       44.8300
high       45.3299
low        44.6100
close      44.8200
volume    12572421
Name: 2022-06-03, dtype: object

In [49]:
test[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float)

open     44.8300
high     45.3299
low      44.6100
close    44.8200
Name: 2022-06-03, dtype: float64

This pulls the average of open, high, low, and close. Using this because we have no idea when in the day the purchase was made.

In [55]:
test[mcconnell.loc[0, 'ticker']].loc[mcconnell.loc[0, 'transaction_date']].drop('volume').astype(float).mean()

44.897475